In [34]:
import pandas as pd
import numpy as np
# Plot the performance
import matplotlib.pyplot as plt
import math
from functools import reduce
import os
from scipy.signal import find_peaks

In [54]:
# Get the current working directory
current_dir = os.getcwd()
# Define the path to the folder in the parent directory
data_folder_path = os.path.join(current_dir, "data")

# Define the file path
# file_name = 'US9L ETF Stock Price History.csv'
# file_name = 'MacroTrends_Data_Download_SPY.csv'
# file_name = 'Amumbo CL2 Historical Data.csv'
# file_name = 'SPXL ETF Stock Price History(1).csv'
file_name = 'spxl_generated.csv'
# file_name = 'SSO ETF Stock Price History.csv'
file_path = os.path.join(data_folder_path, file_name)

# Load the text file into a DataFrame
# Adjust parameters like delimiter, header, and column names as needed
df_full = pd.read_csv(file_path, delimiter=",", header=0)

# Convert the column to a date format
df_full['date'] = pd.to_datetime(df_full['date'], errors='coerce')
df_full = df_full.sort_values(by='date').reset_index(drop=True)
df_full['Price'] = df_full['3x_price'].copy() ## INPUT - specify which column to assign to Price column which will be used for the simulation
# Display the first few rows of the DataFrame
print(df_full)

# Define the file path
file_name = 'Gold Futures Historical Data.csv'
file_path = os.path.join(data_folder_path, file_name)

# Load the text file into a DataFrame
# Adjust parameters like delimiter, header, and column names as needed
df_gold = pd.read_csv(file_path, delimiter=";", header=0)

# Convert the column to a date format
df_gold['date'] = pd.to_datetime(df_gold['date'], errors='coerce')
df_gold = df_gold.sort_values(by='date').reset_index(drop=True)
# Display the first few rows of the DataFrame
df_gold

           date    spy_price      Volume  spy_daily_pct_change  \
0    1991-04-04   379.769989   198120000              0.000000   
1    1991-04-05   375.359985   187410000             -0.011612   
2    1991-04-08   378.660004   138580000              0.008792   
3    1991-04-09   373.559998   169940000             -0.013469   
4    1991-04-10   373.149994   167940000             -0.001098   
...         ...          ...         ...                   ...   
8495 2024-12-24  6040.040039  1757720000              0.011043   
8496 2024-12-26  6037.589844  2904530000             -0.000406   
8497 2024-12-27  5970.839844  3159610000             -0.011056   
8498 2024-12-30  5906.939941  3433250000             -0.010702   
8499 2024-12-31  5881.629883  3128350000             -0.004285   

      2x_daily_pct_change  3x_daily_pct_change  10x_daily_pct_change  \
0                0.000000             0.000000              0.000000   
1               -0.023225            -0.034837             -0.1

,date,Price,Open,High,Low,Vol.,Change %
0,1996-04-01,397.7,398.1,398.1,396.9,26.28K,-0.18%
1,1996-04-02,396.6,397.7,397.7,395.7,32.29K,-0.28%
2,1996-04-03,396.8,396.5,398.3,396.3,18.78K,0.05%
3,1996-04-04,397.6,396.8,397.8,395.7,32.39K,0.20%
4,1996-04-08,400.7,397.8,401.0,397.1,43.70K,0.78%
...,...,...,...,...,...,...,...
7306,2025-02-14,2900.7,2957.3,2964.1,2889.9,228.08K,-1.52%
7307,2025-02-18,2949.0,2895.1,2956.5,2887.6,218.76K,1.67%
7308,2025-02-19,2936.1,2954.1,2964.4,2933.7,159.34K,-0.44%
7309,2025-02-20,2956.1,2949.5,2973.4,2939.3,172.20K,0.68%


In [352]:
# Get the current working directory
current_dir = os.getcwd()
# Define the path to the folder in the parent directory
data_folder_path = os.path.join(current_dir, "data")

# Define the file path
file_name = 'BAMLH0A0HYM2.csv'
file_path = os.path.join(data_folder_path, file_name)

# Load the text file into a DataFrame
# Adjust parameters like delimiter, header, and column names as needed
df_cs = pd.read_csv(file_path, delimiter=",", header=0)

# Convert the column to a date format
df_cs['date'] = pd.to_datetime(df_cs['date'], errors='coerce')
df_cs = df_cs.sort_values(by='date').reset_index(drop=True)
# Display the first few rows of the DataFrame
df_cs = df_cs.dropna()
## INPUT - specify start date for credit spread signal analysis ##
start_date_credit_spread = '1997-01-01'
start_date_credit_spread = '2008-01-01'
###########
df_cs = df_cs[df_cs['date'] >= start_date_credit_spread]
df_cs = df_cs.reset_index(drop=True)
df_cs

,date,BAMLH0A0HYM2
0,2008-01-02,6.10
1,2008-01-03,6.17
2,2008-01-04,6.36
3,2008-01-07,6.45
4,2008-01-08,6.51
...,...,...
4467,2025-02-07,2.67
4468,2025-02-10,2.66
4469,2025-02-11,2.66
4470,2025-02-12,2.65


In [343]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

def get_vesting_periods(df): 
    # Define parameters
    window_var = int(130 * 2.5)
    enter_threshold = -35  # INPUT - Threshold for entering SPXL
    exit_threshold = 30  # INPUT - Threshold for exiting SPXL
    invested_flag = False  # Start with not invested
    entry_date, exit_date = None, None
    enter_list, exit_list = [], []

    # Calculate 330-day EMA
    df["EMA_330"] = df["BAMLH0A0HYM2"].ewm(span=330, adjust=False).mean()

    # Iterate through the DataFrame from window+1 row onwards
    for i in range(window_var, len(df)):
        current_B = df.loc[i, 'BAMLH0A0HYM2']
        current_date = df.loc[i, 'date']
        current_ema_330 = df.loc[i, 'EMA_330']

        if invested_flag:
            if entry_date is not None:
                days_diff = (current_date - entry_date).days
                window = min(window_var, days_diff)
        else:
            if exit_date is not None:
                days_diff = (current_date - exit_date).days
                window = min(window_var, days_diff)
            else:
                window = window_var
            
        # Extract the last 'window' rows including the current row
        window_df = df.iloc[i - window : i]
        
        # Get column B values as a NumPy array
        y = window_df['BAMLH0A0HYM2'].values
        
        # Find peaks (local maxima) and dips (local minima)
        prominence = 0.4
        peaks, _ = find_peaks(y, prominence=prominence)
        dips, _ = find_peaks(-y, prominence=prominence)
        
        # Get the highest peak value and lowest dip value
        highest_peak = max(y[peaks]) if len(peaks) > 0 else None
        lowest_dip = min(y[dips]) if len(dips) > 0 else None
        
        # Calculate percentage difference
        diff_peak = (100 * (current_B - highest_peak) / highest_peak) if highest_peak is not None else None
        diff_dip = (100 * (current_B - lowest_dip) / lowest_dip) if lowest_dip is not None else None

        # Trading logic
        if not invested_flag and diff_peak is not None and diff_peak < enter_threshold:
            invested_flag = True
            entry_date = current_date  # Store entry date
            enter_list.append(entry_date.strftime('%Y-%m-%d'))
    
        elif invested_flag and diff_dip is not None and diff_dip > exit_threshold and current_B > current_ema_330:
            invested_flag = False
            exit_date = current_date  # Store exit date
            print(f"Enter: {entry_date.strftime('%Y-%m-%d')} --> Exit: {exit_date.strftime('%Y-%m-%d')}")
            exit_list.append(exit_date.strftime('%Y-%m-%d'))

    return enter_list, exit_list

# Run the function
enter_list, exit_list = get_vesting_periods(df_cs)
print(enter_list)
print(exit_list)


Enter: 2003-04-04 --> Exit: 2005-04-15
Enter: 2006-04-26 --> Exit: 2007-07-11
Enter: 2009-04-30 --> Exit: 2011-08-04
Enter: 2012-03-14 --> Exit: 2014-09-26
Enter: 2016-07-11 --> Exit: 2018-11-15
Enter: 2019-12-18 --> Exit: 2020-02-27
Enter: 2020-05-22 --> Exit: 2022-02-24
Enter: 2023-07-19 --> Exit: 2024-08-05
['2003-04-04', '2006-04-26', '2009-04-30', '2012-03-14', '2016-07-11', '2019-12-18', '2020-05-22', '2023-07-19']
['2005-04-15', '2007-07-11', '2011-08-04', '2014-09-26', '2018-11-15', '2020-02-27', '2022-02-24', '2024-08-05']


In [345]:
last_date = '2024-12-30' # INPUT - Last date for the analysis 
if len(exit_list) == len(enter_list):
    exit_list[-1] = last_date
else:
    exit_list.append(last_date)
print(exit_list)

['2005-04-15', '2007-07-11', '2011-08-04', '2014-09-26', '2018-11-15', '2020-02-27', '2022-02-24', '2024-12-30']


In [346]:
def find_price_on_date(date, df, col_name):
    price = None
    for i in range(0, 5):
        new_date = pd.to_datetime(date) + pd.Timedelta(days=i)
        try:
            price = df[df['date'] == new_date][col_name].values[0]
            break
        except Exception as e:
            # print(e)
            pass
    return price

def get_drawdown(df, start_date, end_date, date_col, price_col):
    # Ensure date column is in datetime format
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])

    # Filter DataFrame based on date range
    df_filtered = df[(df[date_col] >= start_date) & (df[date_col] <= end_date)].copy()

    if df_filtered.empty:
        return None  # No data available

    # Identify peaks (local maxima) - highest prices before dips
    peaks, _ = find_peaks(df_filtered[price_col])

    # Identify dips (local minima)
    dips, _ = find_peaks(-df_filtered[price_col])  # Negate prices to find valleys

    if len(dips) == 0:
        return None  # No dips found

    # Find the minimum dip (lowest price)
    min_dip_index = dips[np.argmin(df_filtered.iloc[dips][price_col])]
    min_dip_price = df_filtered.iloc[min_dip_index][price_col]

    # Find the highest price before the lowest dip
    valid_peaks = [p for p in peaks if p < min_dip_index]
    if not valid_peaks:
        return None  # No peak before dip

    max_peak_index = max(valid_peaks, key=lambda x: df_filtered.iloc[x][price_col])
    max_peak_price = df_filtered.iloc[max_peak_index][price_col]

    # Calculate percentage drawdown
    percent_drop = ((max_peak_price - min_dip_price) / max_peak_price) * 100

    return {
        "lowest_dip_date": df_filtered.iloc[min_dip_index][date_col],
        "lowest_dip_price": min_dip_price,
        "highest_before_dip_date": df_filtered.iloc[max_peak_index][date_col],
        "highest_before_dip_price": max_peak_price,
        "percent_drop": percent_drop
    }


In [347]:
## INPUT - Starting capital and taxes ##
start = 10000
# tax = 0.26375 # 25% capital gains + 1.375% solli
# tax = 0.1846 # with teilfreistellung for ETF
tax = 0
net_after_tax = 1 - tax

In [348]:
# Initialize variables
P = start
count = 0
price_col = 'Price'  
gold_flag = 0
real_start_date = "1997-01-02"  ##INPUT - start date to use for CAGR calculation
# real_start_date = '2009-04-30' ##INPUT - start date to use for CAGR calculation

# List to store results
results = []

for i in range(0, len(enter_list)):
    try:
        # Invest in Gold
        if gold_flag == 1:
            if count > 0:
                gold_entry_date = exit_list[i-1]
            else:
                gold_entry_date = real_start_date
                
            entry = find_price_on_date(gold_entry_date, df_gold, price_col)
            exit = find_price_on_date(enter_list[i], df_gold, price_col)
            final = P * exit / entry
            
            if final > P:
                profit = final - P
                net = profit * net_after_tax
                pl_ratio = (P + net) / P
                P = P + net
            else:
                pl_ratio = final / P
                P = final
            pl_pct = (pl_ratio - 1) * 100
            ## Get Drawdown
            drawdown_dict = get_drawdown(df_gold, gold_entry_date, enter_list[i], 'date', 'Price')
            max_drawdown = drawdown_dict['percent_drop']
            drawdown_date = drawdown_dict['lowest_dip_date']
            # Store Gold trade
            results.append(["Gold", gold_entry_date, entry, enter_list[i], exit, round(pl_ratio, 2), round(pl_pct, 1), round(max_drawdown, 1), drawdown_date])

        # Invest in SPXL
        entry = find_price_on_date(enter_list[i], df_full, price_col)
        exit = find_price_on_date(exit_list[i], df_full, price_col)
        final = P * exit / entry
        
        if final > P:
            profit = final - P
            net = profit * net_after_tax
            pl_ratio = (P + net) / P
            P = P + net
        else:
            pl_ratio = final / P
            P = final
        pl_pct = (pl_ratio - 1) * 100
        ## Get Drawdown
        drawdown_dict = get_drawdown(df_full, enter_list[i], exit_list[i], 'date', 'Price')
        max_drawdown = drawdown_dict['percent_drop']
        drawdown_date = drawdown_dict['lowest_dip_date']
        # Store SPXL trade
        results.append(["LETF", enter_list[i], entry, exit_list[i], exit, round(pl_ratio, 2), round(pl_pct, 1), round(max_drawdown,1), drawdown_date])

        count += 1
        if count == 1:
            spxl_entry = entry
            start_date = enter_list[i]
        spxl_exit = exit
        end_date = exit_list[i]

    except Exception as e:
        print(f"Error processing trade {i}: {e}")

# Create DataFrame
df_results = pd.DataFrame(results, columns=["Asset", "Entry Date", "Entry Price", "Exit Date", "Exit Price", "P/L Ratio", "P/L %", "Max. drawdown %", "Drawdown date"])

df_results


,Asset,Entry Date,Entry Price,Exit Date,Exit Price,P/L Ratio,P/L %,Max. drawdown %,Drawdown date
0,LETF,2003-04-04,4.395951,2005-04-15,8.830185,2.01,100.9,4.7,2003-04-09
1,LETF,2006-04-26,12.778681,2007-07-11,19.294586,1.51,51.0,21.8,2006-06-13
2,LETF,2009-04-30,1.770349,2011-08-04,3.723950,2.10,110.4,20.7,2009-07-10
3,LETF,2012-03-14,5.120775,2014-09-26,13.280927,2.59,159.4,27.8,2012-06-01
4,LETF,2016-07-11,14.640224,2018-11-15,27.928288,1.91,90.8,14.3,2016-11-04
5,LETF,2019-12-18,41.494656,2020-02-27,33.107478,0.80,-20.2,1.7,2019-12-30
6,LETF,2020-05-22,24.143403,2022-02-24,64.281706,2.66,166.2,20.9,2020-06-11
7,LETF,2023-07-19,64.156653,2024-12-30,129.829885,2.02,102.4,28.6,2023-10-27


In [349]:
print(f'10k to {round(P,2)}')

10k to 1356926.54


In [350]:
def cagr_calc(final_value, initial_value, years):
    return round(100 * ((final_value / initial_value) ** (1 / years) - 1),2)

In [351]:
start_date_to_calc = real_start_date
years_invested = (pd.to_datetime(end_date) - pd.to_datetime(start_date_to_calc)).days / 365.25  # Account for leap years # 
print(f'vested years: {int(years_invested)}')

tax = 0.0 #26375
spx_entry = find_price_on_date(real_start_date, df_full, 'spy_price')
# spx_entry = find_price_on_date(real_start_date, df_full, 'spy_price')
spx_exit = find_price_on_date(end_date, df_full, 'spy_price')
only_spx = round(start * spx_exit/spx_entry,1)
only_spx_profit_net = (only_spx - start) * (1-tax)
only_spx_exit = start + only_spx_profit_net
only_spx_pct = round(100 * ((only_spx_exit/start) - 1),1)
cagr_spx = cagr_calc(only_spx_exit, start, years_invested)
print(f'only SPY (tax: {100*tax}%): {round(spx_entry,2)}->{round(spx_exit,2)} : {only_spx_exit} --> {only_spx_pct}% --> cagr = {cagr_spx}%')

tear_strategy = round(P,1) 
tear_strategy_pct = round(100 * ((tear_strategy/10000)-1),1) 
cagr_tear = cagr_calc(tear_strategy, start, years_invested)
print(f'tear strategy: {tear_strategy} --> {tear_strategy_pct}% --> cagr = {cagr_tear}%')

vested years: 27
only SPY (tax: 0.0%): 737.01->5906.94 : 80147.4 --> 701.5% --> cagr = 7.72%
tear strategy: 1356926.5 --> 13469.3% --> cagr = 19.17%


In [291]:
tax = 0.0 #26375
entry = find_price_on_date(real_start_date, df_full, 'Price') #spxl_entry 
exit = find_price_on_date(end_date, df_full, 'Price') #spxl_exit 
only_spxl = round(start * exit/entry,1)
only_spxl_profit_net = (only_spxl - start) * (1-tax) 
only_spxl_exit = start + only_spxl_profit_net
only_spxl_pct = round(100 * ((only_spxl_exit/start) - 1),1)
cagr_spxl = cagr_calc(only_spxl_exit, start, years_invested)
print(f'only LETF (tax: {100*tax}%): {only_spxl_exit} --> {only_spxl_pct}% --> cagr = {cagr_spxl}%')

only LETF (tax: 0.0%): 212798.1 --> 2028.0% --> cagr = 11.54%


In [179]:
# Get the current working directory
current_dir = os.getcwd()
# Define the path to the folder in the parent directory
data_folder_path = os.path.join(current_dir, "data")

# Define the file path
file_name = 'Gold Futures Historical Data.csv'
file_path = os.path.join(data_folder_path, file_name)

# Load the text file into a DataFrame
# Adjust parameters like delimiter, header, and column names as needed
df_gold = pd.read_csv(file_path, delimiter=";", header=0)

# Convert the column to a date format
df_gold['date'] = pd.to_datetime(df_gold['date'], errors='coerce')
df_gold = df_gold.sort_values(by='date').reset_index(drop=True)
# Display the first few rows of the DataFrame
df_gold

,date,Price,Open,High,Low,Vol.,Change %
0,1996-04-01,397.7,398.1,398.1,396.9,26.28K,-0.18%
1,1996-04-02,396.6,397.7,397.7,395.7,32.29K,-0.28%
2,1996-04-03,396.8,396.5,398.3,396.3,18.78K,0.05%
3,1996-04-04,397.6,396.8,397.8,395.7,32.39K,0.20%
4,1996-04-08,400.7,397.8,401.0,397.1,43.70K,0.78%
...,...,...,...,...,...,...,...
7306,2025-02-14,2900.7,2957.3,2964.1,2889.9,228.08K,-1.52%
7307,2025-02-18,2949.0,2895.1,2956.5,2887.6,218.76K,1.67%
7308,2025-02-19,2936.1,2954.1,2964.4,2933.7,159.34K,-0.44%
7309,2025-02-20,2956.1,2949.5,2973.4,2939.3,172.20K,0.68%


In [21]:
col_name = 'Price'
date_list, spxl_list, gold_list = [], [], []
for date in enter_list:
    spy_3x = find_price_on_date(date, df_full, col_name)
    gold = find_price_on_date(date, df_gold, col_name)
    date_list.append(date)
    spxl_list.append(spy_3x)
    gold_list.append(gold)

for date in exit_list:
    spy_3x = find_price_on_date(date, df_full, col_name)
    gold = find_price_on_date(date, df_gold, col_name)
    date_list.append(date)
    spxl_list.append(spy_3x)
    gold_list.append(gold)


# Create DataFrame
df_to_export = pd.DataFrame({'date': date_list, 'spxl': spxl_list, 'gold': gold_list})
df_to_export
# # Export to CSV
df_to_export.to_csv('output.csv', index=False)

In [42]:
date = "1998-07-02"
# date = "2003-04-04"
spy_3x = find_price_on_date(date, df_full, col_name)
gold = find_price_on_date(date, df_gold, col_name)
print(spy_3x)
print(gold)

833.789528307376
295.0
